In [1]:

# one index
import mediapipe as mp
import cv2
import numpy as np
import tensorflow as tf
import os

# Check if model exists, if not, handle the error gracefully
model_path = 'gesture_model.h5'
if os.path.exists(model_path):
    # Load TF model
    model = tf.keras.models.load_model(model_path)
    model_loaded = True
else:
    print(f"Warning: Model file '{model_path}' not found. Running in detection-only mode.")
    model_loaded = False

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5)

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Function to put text with background
def put_text_with_background(img, text, position, font_scale=0.7, thickness=1, text_color=(255, 255, 255), bg_color=(0, 0, 0)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    
    # Draw background rectangle
    cv2.rectangle(img, position, (position[0] + text_w, position[1] + text_h + 5), bg_color, -1)
    
    # Draw text
    cv2.putText(img, text, (position[0], position[1] + text_h), font, font_scale, text_color, thickness)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    
    # Flip the frame horizontally for a more intuitive mirror view
    frame = cv2.flip(frame, 1)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    h, w, _ = frame.shape
    
    # Draw a title on the frame
    put_text_with_background(frame, "Hand Ability Measurement", (10, 30), 
                            font_scale=1, thickness=2, bg_color=(50, 50, 50))
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            landmarks_px = [(int(x * w), int(y * h), z) for x, y, z in landmarks]

            # Calculate angles for index finger joints
            mcp_angle = calculate_angle(landmarks[0], landmarks[5], landmarks[6])
            pip_angle = calculate_angle(landmarks[5], landmarks[6], landmarks[7])
            dip_angle = calculate_angle(landmarks[6], landmarks[7], landmarks[8])
            
            # Display angles on the frame
            y_pos = 70
            put_text_with_background(frame, f"Index Finger Angles:", (10, y_pos))
            y_pos += 30
            put_text_with_background(frame, f"MCP: {mcp_angle:.1f} degrees", (10, y_pos))
            y_pos += 30
            put_text_with_background(frame, f"PIP: {pip_angle:.1f} degrees", (10, y_pos))
            y_pos += 30
            put_text_with_background(frame, f"DIP: {dip_angle:.1f} degrees", (10, y_pos))
            
            # Only predict if model was successfully loaded
            if model_loaded:
                # Prepare input for TF model (flatten landmarks)
                input_data = np.array(landmarks).flatten().reshape(1, -1)

                # Predict finger ability/gesture class
                prediction = model.predict(input_data, verbose=0)  # Added verbose=0 to suppress output
                pred_class = np.argmax(prediction)
                
                # Display prediction on the frame
                y_pos += 40
                put_text_with_background(frame, f"Predicted Gesture: Class {pred_class}", 
                                        (10, y_pos), bg_color=(0, 100, 0))
    else:
        # Display message when no hand is detected
        put_text_with_background(frame, "No hand detected", (10, 70), 
                                bg_color=(0, 0, 100))
    
    cv2.imshow("Hand Ability Measurement", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:

#for full hand
import mediapipe as mp
import cv2
import numpy as np
import tensorflow as tf
import os

# Check if model exists, if not, handle the error gracefully
model_path = 'gesture_model.h5'
if os.path.exists(model_path):
    # Load TF model
    model = tf.keras.models.load_model(model_path)
    model_loaded = True
else:
    print(f"Warning: Model file '{model_path}' not found. Running in detection-only mode.")
    model_loaded = False

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5)

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Function to put text with background
def put_text_with_background(img, text, position, font_scale=0.7, thickness=1, text_color=(255, 255, 255), bg_color=(0, 0, 0)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    
    # Draw background rectangle
    cv2.rectangle(img, position, (position[0] + text_w, position[1] + text_h + 5), bg_color, -1)
    
    # Draw text
    cv2.putText(img, text, (position[0], position[1] + text_h), font, font_scale, text_color, thickness)

# Define finger landmark indices
# MediaPipe hand landmarks:
# Thumb: 1-4
# Index: 5-8
# Middle: 9-12
# Ring: 13-16
# Pinky: 17-20
finger_names = ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
finger_indices = [
    [1, 2, 3, 4],  # Thumb
    [5, 6, 7, 8],  # Index
    [9, 10, 11, 12],  # Middle
    [13, 14, 15, 16],  # Ring
    [17, 18, 19, 20]   # Pinky
]

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    
    # Flip the frame horizontally for a more intuitive mirror view
    frame = cv2.flip(frame, 1)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    h, w, _ = frame.shape
    
    # Draw a title on the frame
    put_text_with_background(frame, "Hand Ability Measurement", (10, 30), 
                            font_scale=1, thickness=2, bg_color=(50, 50, 50))
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            landmarks_px = [(int(x * w), int(y * h), z) for x, y, z in landmarks]
            
            # Calculate and display angles for all fingers
            y_pos = 70
            
            # Left side for MCP and PIP angles
            left_x = 10
            put_text_with_background(frame, "Finger Angles (degrees):", (left_x, y_pos), bg_color=(50, 50, 50))
            y_pos += 30
            
            # Right side for DIP angles and additional info
            right_x = w - 200
            
            for i, (name, indices) in enumerate(zip(finger_names, finger_indices)):
                # For thumb, we use different reference points
                if name == 'Thumb':
                    # CMC joint (between wrist and thumb base)
                    cmc_angle = calculate_angle(landmarks[0], landmarks[1], landmarks[2])
                    # MCP joint
                    mcp_angle = calculate_angle(landmarks[1], landmarks[2], landmarks[3])
                    # IP joint (thumb has only one interphalangeal joint)
                    ip_angle = calculate_angle(landmarks[2], landmarks[3], landmarks[4])
                    
                    # Display on frame
                    put_text_with_background(frame, f"{name}: CMC={cmc_angle:.1f} MCP={mcp_angle:.1f}", 
                                           (left_x, y_pos), bg_color=(50, 0, 0))
                    put_text_with_background(frame, f"IP={ip_angle:.1f}", 
                                           (right_x, y_pos), bg_color=(50, 0, 0))
                else:
                    # For other fingers, calculate MCP, PIP, and DIP angles
                    # MCP joint
                    mcp_angle = calculate_angle(landmarks[0], landmarks[indices[0]], landmarks[indices[1]])
                    # PIP joint
                    pip_angle = calculate_angle(landmarks[indices[0]], landmarks[indices[1]], landmarks[indices[2]])
                    # DIP joint
                    dip_angle = calculate_angle(landmarks[indices[1]], landmarks[indices[2]], landmarks[indices[3]])
                    
                    # Display on frame
                    put_text_with_background(frame, f"{name}: MCP={mcp_angle:.1f} PIP={pip_angle:.1f}", 
                                           (left_x, y_pos), bg_color=(0, 50, 0))
                    put_text_with_background(frame, f"DIP={dip_angle:.1f}", 
                                           (right_x, y_pos), bg_color=(0, 50, 0))
                
                y_pos += 30
            
            # Only predict if model was successfully loaded
            if model_loaded:
                # Prepare input for TF model (flatten landmarks)
                input_data = np.array(landmarks).flatten().reshape(1, -1)

                # Predict finger ability/gesture class
                prediction = model.predict(input_data, verbose=0)  # Added verbose=0 to suppress output
                pred_class = np.argmax(prediction)
                
                # Display prediction on the frame
                y_pos += 10
                put_text_with_background(frame, f"Predicted Gesture: Class {pred_class}", 
                                        (left_x, y_pos), bg_color=(0, 100, 0))
    else:
        # Display message when no hand is detected
        put_text_with_background(frame, "No hand detected", (10, 70), 
                                bg_color=(0, 0, 100))
    
    cv2.imshow("Hand Ability", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()